<a id='Q0'></a>
<center> <h1> Notebook 2: SRF News Headlines Analysis</h1> </center>
<p style="margin-bottom:1cm;"></p>
<center><strong>Angela Niederberger, 2022</strong></center>
<p style="margin-bottom:1cm;"></p>

<div style="background:#EEEDF5;border-top:0.1cm solid #EF475B;border-bottom:0.1cm solid #EF475B;">
    <div style="margin-left: 0.5cm;margin-top: 0.5cm;margin-bottom: 0.5cm;color:#303030">
        <p><strong>Goal:</strong> In this notebook I perform a preliminary explorative analysis of the new headlines data scraped from the SRF website.</p>
        <strong> Outline:</strong>
        <a id='P0' name="P0"></a>
        <ol>
            <li> <a style="color:#303030" href='#I'>Introduction </a> </li>
            <li> <a style="color:#303030" href='#SU'>Set up</a></li>
            <li> <a style="color:#303030" href='#P1'>Explorative Analysis</a></li>
            <li> <a style="color:#303030" href='#P2'>Saving the Data</a></li>
            <li> <a style="color:#303030" href='#P3'>Code Refactor</a></li>
            <li> <a style="color:#303030" href='#CL'>Conclusion</a></li>
        </ol>
        <strong>Keywords:</strong> EDA, BeautifulSoup, AWS S3
    </div>
</div>
</nav>

<a id='I' name="I"></a>
## [Introduction](#P0)

In the first notebook I documented how to create a dataset on SRF news headlines using an automated webscraping script on AWS. Now I will have a look at the data I collected to monitor if the scraping is working as expected and to gain some first insights into the data itself.


<a id='SU' name="SU"></a>
## [Set up](#P0)

I created a virtual environment and installed Python 3.10.4 in it for this project. Below are the specifics on all the packages I used. I've also summarized all of this information in the requirements file.

### Packages

In [100]:
import pandas as pd  # Version 1.4.2
import s3fs  # Version 2022.5.0

### Magic Commands

In [3]:
%config Completer.use_jedi = False

### Global Variables

In [99]:
s3_folder_path = "s3://srf-news-snippets/data/"

### Reading in the Data

In [93]:
# Initialize the file system and load data
fs = s3fs.S3FileSystem(anon=False)
dataframe_list = [pd.read_csv(f"s3://{file}", index_col=0) for file in fs.ls(s3_folder_path)[1:]]
headlines_df = pd.concat(dataframe_list, ignore_index=True)

# Clean data and add features
headlines_df["time_published"] = pd.to_datetime(headlines_df["time_published"])
headlines_df["date"] = headlines_df["time_published"].dt.date
headlines_df["weekday"] = headlines_df["time_published"].dt.strftime("%A")

headlines_df.head()

,time_published,kicker,title,lead,date,weekday
0,2022-06-01 22:57:00+02:00,3:1-Erfolg in Schottland,Entschlossene Ukraine darf weiter von der WM t...,In einem hochemotionalen Playoff-Halbfinal der...,2022-06-01,Wednesday
1,2022-06-01 22:09:00+02:00,Weg zur Energiewende,Energiegesetze: Wie viel Zwang ist möglich?,Aargau und Solothurn überarbeiten ihre Energie...,2022-06-01,Wednesday
2,2022-06-01 21:37:00+02:00,Repression in der Türkei,Erdogan fürchtet um seine Macht,"Wer nicht für ihn ist, ist gegen ihn: Erdogan ...",2022-06-01,Wednesday
3,2022-06-01 21:32:00+02:00,Depp vs. Heard,Johnny Depp gewinnt Verleumdungsprozess gegen ...,Die Geschworenen haben dem Hollywood-Star eine...,2022-06-01,Wednesday
4,2022-06-01 21:08:00+02:00,Paris: Viertelfinals Männer,Cilic auch von Rublew nicht zu stoppen,Marin Cilic schlägt Andrej Rublew in 5 Sätzen ...,2022-06-01,Wednesday


In [94]:
headlines_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110 entries, 0 to 109
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype                                
---  ------          --------------  -----                                
 0   time_published  110 non-null    datetime64[ns, pytz.FixedOffset(120)]
 1   kicker          110 non-null    object                               
 2   title           110 non-null    object                               
 3   lead            110 non-null    object                               
 4   date            110 non-null    object                               
 5   weekday         110 non-null    object                               
dtypes: datetime64[ns, pytz.FixedOffset(120)](1), object(5)
memory usage: 5.3+ KB


<a id='SU' name="SU"></a>
## [Explorative Analysis](#P0)

In [95]:
headlines_df[headlines_df["title"].str.contains("Queen")]

,time_published,kicker,title,lead,date,weekday
17,2022-06-01 15:42:00+02:00,Königin der Superlative,7 Fakten zum 70. Thronjubiläum von Queen Eliza...,"Pokale, Päpste, Ponys: Elizabeth II. hat viel ...",2022-06-01,Wednesday
24,2022-06-01 11:14:00+02:00,70. Thronjubiläum,«Die Queen ist ein lebendes Monument»,Seit 70 Jahren sitzt Queen Elizabeth auf dem b...,2022-06-01,Wednesday
32,2022-06-02 21:35:00+02:00,70. Thronjubiläum,Queen sagt Teilnahme an Dankgottesdienst in Lo...,Die 96-jährige Monarchin verpasst die Zeremoni...,2022-06-02,Thursday
48,2022-06-02 15:58:00+02:00,70. Thronjubiläum der Queen,Grosse Parade zu Ehren der Queen – die Bilder,"Kanonenschüsse, Marschmusik und Formationsflug...",2022-06-02,Thursday
57,2022-06-02 10:48:00+02:00,Amtsälteste Monarchin der Welt,Grossbritannien feiert das Thronjubiläum der Q...,Mit der Militärparade «Trooping the Colour» st...,2022-06-02,Thursday
83,2022-06-03 13:23:00+02:00,70. Thronjubiläum der Queen,Das grosse Queen-Quiz: Was wissen Sie über Eli...,1952 hat Queen Elizabeth II. den britischen Th...,2022-06-03,Friday
102,2022-06-04 17:56:00+02:00,Queen Elizabeths Thronjubiläum,Strassenparty zu Ehren der Queen: zu Besuch be...,"Das 70. Thronjubiläum wird mit Paraden, Gottes...",2022-06-04,Saturday
107,2022-06-04 11:47:00+02:00,Debatte um Monarchie,Nicht alle feiern 70. Thronjubiläum der Queen,Britische Aktivisten hoffen auf Ende der Monar...,2022-06-04,Saturday


In [98]:
headlines_df[headlines_df["kicker"].str.contains("Ukraine")]

,time_published,kicker,title,lead,date,weekday
20,2022-06-01 14:33:00+02:00,Krieg in der Ukraine,Waffen für die Ukraine: «Scholz versucht den B...,Nach langem Zögern sagt Scholz der Ukraine wei...,2022-06-01,Wednesday
34,2022-06-02 20:25:00+02:00,Angriff auf die Ukraine,Anne Applebaum: «Putin muss vom Geld abgeschni...,Die preisgekrönte Journalistin und Historikeri...,2022-06-02,Thursday
43,2022-06-02 16:45:00+02:00,100 Tage Krieg in der Ukraine,Ukraine-Krieg: Das hat sich für die Schweiz ve...,Fast 100 Tage dauert der Krieg bereits. Die Au...,2022-06-02,Thursday
72,2022-06-03 19:02:00+02:00,Waffen für die Ukraine?,Schweiz lockert ihre Praxis in der Waffenfrage...,Es bleibt dabei: Deutschland darf Munition aus...,2022-06-03,Friday
77,2022-06-03 16:38:00+02:00,100 Tage Ukraine-Krieg,Kiew – wo der Krieg als Gespenst durch die Str...,Viele haben die Stadt verlassen. Wer geblieben...,2022-06-03,Friday
86,2022-06-03 11:28:00+02:00,Krieg in der Ukraine,Militärexperte: «US-Waffen für Ukraine könnten...,Die USA und Deutschland liefern moderne Waffen...,2022-06-03,Friday
103,2022-06-04 16:37:00+02:00,Flüchtlinge aus der Ukraine,Kantone brauchen dringend mehr Betreuungsperso...,Neun Kantone mussten 400 neue Stellen für die ...,2022-06-04,Saturday
108,2022-06-04 08:19:00+02:00,Moldawien und der Ukrainekrieg,"«Wenn die Russen aus dem Wachturm schauen, sin...",Der Donauhafen von Giurgiulesti ist der einzig...,2022-06-04,Saturday


In [96]:
headlines_df.groupby("date")["date"].count()

date
2022-06-01    29
2022-06-02    36
2022-06-03    30
2022-06-04    15
Name: date, dtype: int64

<a id='CL'></a>
## [Conclusion](#P0)



<div style="border-top:0.1cm solid #EF475B"></div>
    <strong><a href='#Q0'><div style="text-align: right"> <h3>End of this Notebook.</h3></div></a></strong>